# MNIST For ML Beginners: The Bayesian way
(c) 2017 Sreekumar Thaithara Balan, Alpha-I 

This tutorial is intended for readers who are new to machine learning, TensorFlow and **Bayesian Methods**. Our intension is to created a Bayesian companion to the well known getting [started example](https://www.tensorflow.org/get_started/mnist/beginners) in TensorFlow. The readers are highly recommened to read that tutorial as a prerequisite to this blog. 

## About this tutorial
In thi tutorial, you will learn about

+ Construct a Bayesian model for classification of MNIST images
+ See how the Bayesain networks can quantify uncertainties in prediction

*For those who are eager to see why we care about uncertainites, scroll down to the bottom of this blog where we input the image of the letter "A" and ask our model to classify it. With Bayesian models we can how confident we are about our predictions!*